# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [14]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [18]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script('return document.body.scrollHeight')

while True:
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(SCROLL_PAUSE_TIME)
    new_height= browser.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        print("到達頁面最底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面最底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [19]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [20]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 7月景氣分數略減1.16分　台經院：3大經濟隱憂需觀察

▲台經院公布7月分景氣燈號信號值。（圖／台經院提供）
記者余弦妙／台北報導
台經院表示，隨著Delta病毒擴散，包括東南亞、日、美等疫情再度增溫，影響部分國家復甦腳步，不過台灣受惠於全球需求回升及主要國家擴大基礎建設，出口、外銷訂單及生產等經濟指標仍有雙位數成長，惟部分國家疫情復燃，導致漲幅已明顯出現縮小，影響需求面及原物料投入面指標表現。
儘管本土疫情趨緩，但台股在美國疫情升溫及半導體大廠法說會內容不如預期等因素，呈先漲後跌態勢，影響經營環境面指標表現。
此次黃紅燈的分數低於前幾個月的表現，主要是受到去年比較基期逐月墊高情況，加上近期美、日疫情升溫、東南亞疫情加劇，衝擊了全球經濟復甦及產業供應鏈的步調，進而使得生產、外銷訂單等實體經濟指標的漲幅不若之前強勁。
展望未來，國內自7月27日起調降疫情警戒標準至第二級，顯示本土疫情已呈現趨緩，政府在10月份將推出五倍券、加上各縣市政府及民間企業加碼，可促進消費需求增溫，隨著購買力道持續，將有助於內需產業的表現。
----------------------------------------------------------------------

[1] 「麟洋」卡兩卡面曝光！土銀限量發行　目標半年內5萬張

▲「麟洋」卡來了。（圖／記者陳依旻攝）
記者陳依旻／台北報導
球迷許願的「麟洋」羽球認同卡來了！土銀今（31）日邀請自家羽球金牌王齊麟、李洋，與董事長謝娟娟、總經理何英明一起出席揭卡儀式，董事長謝娟娟會後受訪時強調，目標半年內發卡5萬張。
謝娟娟指出，麟洋奧運摘金滿月 ，滿月日子推出認同卡，相信大家都記憶猶新，特別是羽球終局線上的經典畫面，成為所有商品套用圖騰，網路上敲碗推出，在千呼萬喚之下，今日終於與大家見面了，回想誕生的過程，從發想到設計、得到東奧、VISA授權、審核「有很多波折」。
謝娟娟強調，此張信用卡紀念性除了有東奧授權，還有材質使用環保相關，符合節能減碳、ESG，持卡人每一筆刷卡都會回饋到社會角落，除了會提撥消費額的0.25%計算回饋金，協助國內羽球運動發展及培育優秀人才，另外，只要新申辦且

圖、文／鏡週刊
全盛時期總資產高達600億元的老牌食品大廠掬水軒，在台灣可說無人不知、無人不曉，但1999年創辦人柯家第二代柯德勝病逝之後，遺孀柯陳幸佳及第三代兄妹柯富元、柯俊材、柯玫岑等人，就陷入爭產互告、掏空公司的紛擾，至今已超過20年，還衍生出50多件官司。如今又有大股東後代出面向本刊投訴，指控柯家欺騙他們，法院也未公平處理！
▲掬水軒2代董事長柯德勝過世後，遺孀及子女為了爭產對簿公堂。（圖／鏡週刊提供，下同）
但柯德勝病逝後，其妻柯陳幸佳涉嫌隱瞞丈夫死訊，並偽造遺囑，讓自己當上董事長，將公司資產以各種方式轉移到自己名下，而第三代董事長柯富元接手後，還為此控告母親，最後柯陳幸佳被依偽造文書、逃漏稅等罪嫌起訴並通緝，從此滯留日本不再回台。
至於柯富元則因公司經營不善、欠缺資金，竟利用桃園平鎮廠區轉型購物中心的開發案，吸金9億多元，雖然短暫解決資金缺口，卻讓掬水軒背負了更多債務，最後也因背信、違反《銀行法》等罪名遭起訴、判刑。
不料，柯富元繳交200萬元保釋金後，竟棄保潛逃至中國，他的弟弟柯俊材也避居美國，掬水軒這個老字號也逐漸被市場遺忘。
L小姐表示：「就算如此，掬水軒公司目前仍登記在案，有新的董事長接任，更有上百名小股東，我們呼籲法院認真看待相關訴訟案，不要胡亂賤賣掬水軒的公司資產，侵害我們這些小股東的權益！」
----------------------------------------------------------------------

[15] 彭博社：蘋果執行長庫克最快2025年退休、將留「新產品」系列

▲蘋果執行長庫克可能最快在2025年退休。（圖／取自9to5mac）
記者陳俐穎／綜合報導
在過去的幾年裡，關於誰將繼庫克之後接任蘋果 CEO 的猜測頗多，而最近，庫克本人也提到，他很可能在10 年後就不會在蘋果工作。而彭博社記者Mark Gurman也對此進行推測，認為蘋果內部希望庫克可以留下「新產品」系列。
在最新一期的 Power On 時事通訊中，彭博社的Mark Gurman 對蘋果 CEO 職位的未來進行了推測，並表示蘋果內部是希望庫克能留下來開發一個更重要的新產品類別。Mark Gurman認為，庫克特別關注蘋果的擴增實境眼鏡。AR/VR相關的硬體新產品很可能是庫克最終要發展的產品類別。
Gurman還重申，蘋果計劃在 20

▲2021年國際大學校院會展城市行銷競賽首推「線上虛擬化」，即日起至9月6日開放報名。(圖／貿協提供)
記者張佩芬／台北報導
國際大學校院會展城市行銷競賽於睽違一年強勢回歸！貿協表示，競賽提供國內外青年學子會展實作及國際交流機會，今年競賽即日起開放報名至9月6日止，競賽結果公布日期為11月12日，外貿協會邀請國內外大學校院師生組隊報名，把握機會以免錯過！
國際大學校院會展城市行銷競賽由經濟部國際貿易局及外貿協會主辦，自2009年開辦至今以逾十年，歷年參賽隊伍包括來自臺灣、日本、泰國、越南、印尼、印度、馬來西亞、新加坡、中國及德國等，並以「為所選城市規畫會展行銷企劃」為競賽主軸，由國內外大學校院學生來臺辦理實體展覽並宣傳各國城市，達到會展國際交流的目的。
貿協表示，因應後疫情會展趨勢及會展新常態，今年競賽首度轉至「虛擬會展平台」，並以「線上國際城市行銷與展覽OMO(Online-Merge-Offline)」為競賽主題，讓學生為所選城市設計線上會展行銷方案並展出，培養未來會展產業人才虛實整合與行銷能力。
主辦單位將提供每隊伍免費虛擬會展平台帳號，內含5個虛擬空間以及100個多媒體標籤，激發參賽者發揮創意，創建獨一無二的展覽空間。此外，活動亦由虛擬會展平台廠商提供賽前免費線上工作坊中英文課程，搭配素材練習包以及中英文教學影片，協助國內外學生不須具備3D設計背景或相關經驗，皆能夠輕鬆上手虛擬展覽，掌握會展科技新趨勢。
此次競賽獎項涵蓋行銷企畫書獎、虛擬展覽與展攤設計獎、英文導覽獎，每競賽項目分別提供冠軍獎金新臺幣2萬元及獎盃乙座、亞軍獎金1.5萬元及獎狀一紙、季軍1萬元及獎狀一紙；將於9月10日公告競賽入圍隊伍，各隊伍須於10月15日繳交英文行銷企畫書，並於11月1日上架虛擬展攤。
最終由國內外會展專家上線瀏覽評分各隊伍虛擬展攤，同時，展攤將開放予世界各地民眾上線瀏覽、互動，讓全世界看見你的城市！
----------------------------------------------------------------------

[28] 團圓網購過中秋！逾400家電商商品限量下殺　還有大獎等你抽

▲經濟部商業司中秋網購節大促活動。（圖／中華民國無店面零售商業同業公會提供）
記者李春棉／綜合報導
中秋節連假即將到來，經濟部商業司帶領財團法人資訊工業策進會、中華

▲財信傳媒董事長謝金河。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
財信傳媒董事長謝金河今（30）日在臉書指出，在中國監管機構宣布不得有看空中國經濟及金融市場言論之後，先前曾發表過不再投資中國相關資產的投資大咖，最近又紛紛改口，最具代表性的是女股神Cathie Wood，及軟體銀行的孫正義。
謝金河提到，女股神前幾天悄悄買入16.49股（編按，應是164,889股）京東，使得京東當天盤中一度大漲11%，一個多月前，Ark幾乎出清所有的中概股，她也呼籲大家要小心謹慎，不過，最近她說：長遠來說，對中國樂觀多於悲觀。比較明顯的是，原來表示對中國投資將保持謹慎，直到他們能夠判斷法規的深度和程度，但沒有多久，孫正義就跳出來說自己的發言被誤解，會繼續投資中國市場。孫正義投資一億美元在阿里，滾成千億美元鉅富，難怪他說馬雲眼中充滿著動物般的光芒，現在馬雲有難，孫正義在中國不再虎虎生威。
至於最熱愛中國的橋水投資創辦人Darlio Ray則仍大力為習近平的共同富裕背書，他的發言，現在被懷疑拿了很多中國好項目及利益。也許他們都有不同的立場與角度，或者是受到一些壓力，或是深愛中國，但都改口表態了。
謝金河指出，這一輪，中概股跌得旣深且重，像補教股新東方一口氣跌91%，好未來跌95%，騰訊跌46.8%，阿里巴巴跌51%，美團跌60%，中國快手跌86%，恆大汽車跌90.5%，嗶哩嗶哩跌53.6%⋯⋯理論上，好的企業遇到不尋常的利空下挫，應該大膽逢低買進才對，阿里，騰訊，美團等，都是中國最有競爭力的企業，股價大跌後，應該是買點才對。不過重點在中國未來的政策走向。
謝金河表示，習近平的共同富裕源自1953年12月的毛澤東講話，不久就有大躍進，文化大革命。這次習近平把共同富裕作為中國共產黨的第二個百年奮鬥目標，建構一次分配，二次分配及三次分配，形成中間大，兩頭小的橄欖型分配結構。
謝金河指出，一次分配是按照市場效率原則進行，這是少數人先富。二次分配是政府主導的收入轉移，因此要加大稅負，社保，轉移支付等調配。三次分配是富者自願慈善捐款，接下來大企業一定會加大力道捐錢，最具代表性的是騰訊，今年上半年騰訊大約賺900億人民幣，這次分兩次捐1000億人民幣，名義是「可持續社會價值創新」，和「共同富裕專項計劃」，如果騰訊每年捐款千億人民幣，未來恐怕變成非盈利事業，股價評價基礎也會不一樣，這是投資中國最大

▲逢甲大學知名度高，周邊學生套房租賃市場較不受少子化影響。（圖／業者提供）
記者林裕豐／台北報導
股市走了1年多頭，不少投資人選擇獲利了結，「由股轉房」趨勢逐漸明顯，其中以帶有租金效益的房屋標的最受青睞，專家分析，「適合獨居的出租套房」及「精緻化的學生套房」是近來熱門選項。
《地產詹哥老實說》S2EP27／有錢人想得跟你不一樣！疫情期照進貨　旅館、廠辦交易頻傳 
過去幾年，店面、商辦等商用不動產，因租金投報高、資金週轉快，成為置產族偏好的選項，不過近來趨勢悄悄轉變，好出租的住宅標的才是王道。
陳佩君分析，近期的租屋者大多有防疫考量，偏好有獨立衛浴的套房，過往在租賃市場有一席之地的雅房產品，租客接受度已下降。
另外，隨著大學開學的腳步逼近，不少大學生也開始找租屋，「現在的父母因為孩子生得少，大多願意付出較高的租金，讓孩子有較好的居住品質，這也讓租屋市場朝向套房化、精緻化發展。」
對於想投資套房產品者，陳佩君建議，要掌握3大原則。
首先，地點絕對要挑對，尤其是少子化時代，許多大學都招生不易，所以要鎖定老牌大學周遭，最好附近還是成熟的辦公商圈，這樣會讓租客來源更多元穩定，例如台中的逢甲、東海周邊，就是不敗的選擇。
其次，要考慮法規需求，包含消防、安全與管理等，都要綜合評估，還得注意屋齡別太老舊，太舊的房子，不僅在租賃市場競爭力低，也會直接影響未來轉售脫手時的增值空間。
第三，財務槓桿要掌握好，陳佩君提醒，一般來說，套房在銀行貸款時會比較受限，如果總坪數低於15坪的，通常沒辦法貸到7成，還得預留一些裝修費用，自有資金需準備充足。
▲專家觀察，現在的父母因為孩子生得少，大多願意付出較高的租金，讓孩子有較好的居住品質。（圖／記者林裕豐攝）
陳佩君認為，套房或小宅產品算是進可攻、退可守，畢竟現在不婚或不生族愈來愈多，老年獨居者也不少，即便是有生小孩的小家庭，也很容易因為房價考量而選小坪數產品，長期來看，套房或小宅產品還是有其市場競爭力。
《地產詹哥老實說》S2EP27／有錢人想得跟你不一樣！疫情期照進貨　旅館、廠辦交易頻傳 
----------------------------------------------------------------------

[50] 陸證監會點名「假私募」亂象　預告將推出「私募基金條例」

記者蔡紹堅／綜合報導
大陸證監會主席易會滿30

▲台灣高鐵今（30）日發行可持續發展債券，發行總金額達10億元。（圖／記者姜國輝攝）
記者林淑慧／台北報導
企業推動 ESG（環境、社會、公司治理）正夯，台灣高鐵今（30）日表示，首度發行可持續發展債券，發行金額新臺幣10億元，期限為3年期、採固定年利率0.3%，為台灣高鐵公司首次發行可持續發展債券，亦是國內交通運輸業第一次發行。所募集資金將全數用於綠色投資及社會效益投資計畫。
所謂的可持續發展債券，指的是發行人將所募得資金，全部用於綠色投資計畫及社會效益投資計畫或其相關放款的融資工具，依據櫃買中心規定申請；依據統計，截至目前國內共發行五檔可持續發展債券，發行總金額達新台幣476.38億元。
▲台灣高鐵董事長江耀宗。（圖／高鐵提供）
台灣高鐵表示，本次發行的可持續發展債券為今年度第2期無擔保普通公司債，募集資金將用於支應「建置燕巢總機廠自動化設備」及「車站旅客資訊系統更新」兩案工程款項。其中，「建置燕巢總機廠自動化設備」係屬綠色投資計畫（污染防治與控制），以達成減少汙染物、保護人權及環境之目標。
其次，「車站旅客資訊系統更新」則屬社會效益投資計畫（可負擔的基礎生活設施），透過更新車站旅客資訊系統，即時提供旅客列車班表、氣象資訊及重大、緊急事件之資訊，同時結合廣播系統，提供訊息警示與人員或物品之協尋功能，提升旅客搭乘高鐵之運輸安全及便利性。
----------------------------------------------------------------------

[59] iPhone 13擬推消光黑、青銅色！外媒統整新機「5大特點」　麥克風更強大

記者吳珍儀／綜合報導
iPhone 13即將於下月發布，外媒預估iPhone 13將會帶來設計、功能和相機等重大升級。隨著更多關於iPhone 13的傳言陸續傳出，外媒MacRumors根據知名爆料者馬克斯·溫巴赫（Max Weinbach）所提出，列出5項最有可能出現的功能與更新。
消光黑和青銅色iPhone 13
蘋果在每年的新iPhone加入各種新色選擇，透過不同顏色也可確保消費者更容易區分新機款與舊機款。而在今年3月曾傳出，蘋果計劃推出‌消光黑的iPhone 13‌ Pro系列。傳言指出，當前的曜石黑將調整成更深、更黑的色調，並且具有消光黑色表面。除了‌消光黑，另一色則是青銅色，與當前的金色有

▲鋼鐵龍頭中鋼集團今（30）日舉行股東會，由董事長翁朝棟主持。（圖／中鋼提供）
記者林淑慧／台北報導
鋼鐵龍頭中鋼今（30）日舉行股東會，董事長翁朝棟指出，中鋼股東人數達108萬人，位居上市公司股東數之冠，且為國內年輕人選股排名第二名；受惠推動鋼鐵高值化，中鋼今年全年稅前盈餘可望達建廠50年來最佳，在國際鋼市持續看旺下，也向股東掛保證中鋼明年營運會比今年好很多。
今年為中鋼建廠五十周年，依據財報，中鋼2020年每股盈餘（EPS)為0.05元，今日股東會通過發放現金股利0.3元，配發率達600%，並以「熊愛臺灣棘輪起子工具組」別具 巧思的紀念品致贈股東，感謝股東堅定的支持。
▲中鋼今年贈送「熊愛台灣棘輪起子工具組」作為股東會紀念品。（圖／中鋼提供）
翁朝棟指出，碳中和、碳稅等挑戰促成「高鋼價時代來臨」，中鋼領先布局綠能產業、水資源技術，深化永續根基，創造更大鋼鐵產業鏈價值，讓中鋼贏在未來，以回報股東的支持也為臺灣產業、經濟及社會發展貢獻心力。
為奠基下一個50年永續發展，中鋼去年1月宣示以「高值化精緻鋼廠」以及「發展綠能產業」為發展雙主軸，今年將營運重心擺在推動 「雙主軸」各項行動方案，及做好節能減碳核心重點工作。
綠能產業方面，中鋼光能公司在集團公司廠房屋頂建置84.8MW太陽光電系統，年發電量1.02億度，每年創造超過4億元售電收入；中鋼投資投資興達海洋基礎公司，從事離岸風電水下基礎生產，在今年7月甫完成第一座水下基礎國產化里程碑。此外，積極發展綠電，結合丹麥哥本哈根基礎建設基金(CIP)投入風場開發，預計113年完成中能29號風場300MW建置商轉。
在節能減碳方面，中鋼超前部署導入再生水及發展海水淡化技術，於107年導入鳳山溪都污再生水，111年再導入臨海再生水廠，屆時每日再生水用量將達6.1萬噸，約占整體補充用水半量。中鋼並於今年3月完成海淡實驗產線，目標兩年內開發成本降至每度25元製程。
展望未來，翁朝棟表示，面對既長又遠的轉型升級之路，中鋼秉持「天道酬勤」、「勝者尤須繫盔緒」 的精神　不因勝利而得意忘形，不因成功而鬆懈怠慢，堅守本心、穩健踏實，持續精進，不斷攀登高峰。
----------------------------------------------------------------------

[69] 買法拍撿便宜　高雄市價打6折最划算

▲Google Pixel Buds A-Series 開箱實測。（影片／記者洪聖壹攝、製作）
特約記者洪聖壹／綜合報導
針對已經霸榜多年的蘋果 AirPods，Google 並沒有選擇抄襲或者追隨，用售價新台幣 3290 元，推出 Pixel Buds A-Series，重新調整音質與 Google 助理（Google Assistant）相關應用，讓耳機成為虛擬貼身助理，實測過程中，分享初步體驗耳機實際感受。
▲Google Pixel Buds A-Series 無線耳機售價 3290元、採用 12mm單體、藍牙5.0傳輸。（圖／記者洪聖壹攝）
＊開箱：
原廠 Google Pixel Buds A-Series 耳機外盒盒裝提供最新的 Google Logo，台灣版本僅提供「就是白」，盒裝外面有雷射標籤，盒裝內提供耳機、充電盒、Type-A轉Type-C傳輸線跟大中小三種不同的耳塞。
▲原廠盒裝提供耳機、Type-A 轉 Type-C 傳輸線跟三種不同尺寸的耳塞套。（圖／記者洪聖壹攝）
＊外觀：
Google Pixel Buds A-Series 外型就像一顆蛋，雖然叫「就是白」，不過有點像是白色的雞蛋殼，四周周圍是橢圓形、兩面是平整的，實測重量 54.5 克，單耳耳機重量僅 5 克，機身外設計了小磁鐵，放回盒裝時會吸附住充電盒、不容易掉落，耳殼外側有個弧形的穩定器，扣接之後增加入耳的穩定性之外，在收入盒中也不容易掉出來。
▲▼Google Pixel Buds A-Series 充電盒外型採用蛋型設計、手感輕巧容易收納。（圖／記者洪聖壹攝）
在設計方面，外觀是帶有一點點灰藍色的白色，充電盒跟耳機都是霧面塑料材質，摸起來不會沾染指紋，蛋殼內是灰色內襯延伸出來作為上蓋的緩衝，開蓋跟關起來的感覺有點像是 AirPods，整體設計質感算是中上水平。
＊實測連線三大實用功能：
Google Pixel Buds A-Series 採用的是藍牙5.0傳輸技術，與近兩年市場上多數藍牙5.0 以上傳輸規範的真無線藍牙耳機同樣是配對容易、隨開即連，觸控手勢也都是大同小異，不過 Google 做得更多，以下三點為記者實際感受時體驗的感覺：
▲Google Pixel Buds A-Series 耳機可自體發聲協助用戶尋找。（圖／記者洪聖壹攝）
＊耳機音質：
Google P

記者吳珍儀／綜合報導
美聯準會（Fed）發表鴿派言論，市場疑慮削減，美股四大指數上周五收紅。台股今（30）日以17244點開出，上漲34點，稍後漲幅擴大至近百點。
蘋概三王中，台積電（2330）今日以602元開出，上漲3元；大立光（3008）以2645元開出，上漲15元；鴻海（2317）則以108.5元開出，上漲0.5元。
今航運、電子股齊漲，航運股部分，陽明、長榮上漲逾1%；萬海上漲逾2％。
美股道瓊工業指數上漲242.68點，或0.69%，收35213.12點；標普 500 指數上漲39.37點，或0.88%，收4509.37點；那斯達綜合指數上漲183.69點，或1.23%，收15129.5點；費城半導體指數上漲66.07點，或1.96%，收3436.45點。
----------------------------------------------------------------------

[89] 北市核心價值燙金！大老闆低調入主正南京東路地王

▲合建大案「南京御邸」以正南京東路地王之姿受到矚目，甫開案不久，即迎來銷售佳績。
圖、文／南京御邸提供
不受疫情影響，高端豪宅買氣持續增溫！市場普遍認為，高端買家因手中資金豐沛，且低利率環境有利於房市表現，帶動豪宅市場交易熱絡，光是今年上半年，南京東路、敦化南、北路等傳統豪宅聚落，就有不少指標案陸續登場，合建大案「南京御邸」，以正南京東路地王之姿受到矚目，該案原為傳產、醫療、飯店、建設等五大家族的發跡起家厝，隱富聚落血統純正，吸引不少重視風水的高端買家入主，甫開案不久，即迎來銷售佳績。
負責現場銷售的專案經理劉鎮瑋表示，有「台北華爾街」之稱的南京東路，自60年代起，見證台灣一路向上起飛的經濟發展史，在台股大榮景時期，大型證券機構、銀行、商辦大樓、百貨、飯店比肩林立，寫下一頁頁南京東路輝煌的傳奇故事，時至今日，這條金融大道仍舊是執掌台北經貿與交通樞紐地位的黃金軸線。
▲本案緊鄰捷運南京復興站與松江南京站，城市核心樞紐的優越位置，掌握台北交通動脈。
▲直通松山機場國門的敦北大道，3000億都更開發量能外溢南京黃金軸線，敦北豪辦與南京金融兩大商圈聯手，將再創城市發展新高峰。
▲「南京御邸」樓高20層，居高臨下、視野遼闊，廣納大台北360度城市風景。
------------------------------